In [9]:
import os
import sys
import pyodbc
import numpy as np
import operator
import pandas as pd
import time
import datetime
import clv_scoring_functions
##(make sure our CLV scoring functions script is accessible)
import glob
from dateutil.relativedelta import relativedelta
import sklearn as sklearn
from sklearn import metrics
from scipy.optimize import minimize, rosen, rosen_der


In [117]:
def SQL_Query(start_date, end_date, UID, password, CUST_LIST ):

    
#Format our date inputs  
    i = str(start_date)
    try:
        dt_start = datetime.datetime.strptime(i, '%Y/%m/%d')
    except ValueError:
        print("Incorrect start format")

    i = str(end_date)    
    try:
        dt_end = datetime.datetime.strptime(i, '%Y/%m/%d')
    except ValueError:
        print("Incorrect start format")

#Formatting the dates for SQL 
    dt_start = str(dt_start) + str('.000')
    dt_end = str(dt_end) + str('.000')
    
#Establish our connection to the server 
    conn_string = 'DRIVER=/usr/local/lib/libtdsodbc.so;SERVER=wnj-datasizesql;PORT=1433;DATABASE=Data Analytics;UID={};PWD={}'.format(UID,password)
    conn = pyodbc.connect(conn_string)

#Run our sql query

    query = ("""
    SELECT b.customer_id,
    round(b.recency + b.frequency/10 + b.AmountPerWeek/100, 2)AS RFM,
    b.recency, b.frequency, b.T, 
    b.order_count,
    b.total_amount,
    b.FirstOrderDate, b.LastOrderDate
    FROM
      (

        SELECT
          a.customer_id,
          a.order_count,
          a.recency,
          a.frequency,
          a.T,
          round(a.total_amount /isnull(NULLIF(a.frequency, 0), 300), 2)AS AmountPerWeek,
          a.total_amount,
          a.FirstOrderDate,
          a.LastOrderDate
        FROM
          (
            SELECT
              cbs.customer_identifier AS customer_id,
              count(DISTINCT cast(DATEPART(WK, td.transaction_date)AS VARCHAR)+cast(year(td.transaction_date)AS VARCHAR))- 1
                AS frequency,
              datediff(WEEK,MIN(td.transaction_date),MAX(
                  td.transaction_date))AS recency,
              datediff(WEEK,MIN(td.transaction_date),
                       getdate())    AS T,
              count(distinct(td.av_transaction_id))as order_count,
              MIN(
                  td.transaction_date)  AS FirstOrderDate,
              MAX(
                  td.transaction_date)  AS LastOrderDate,
              round(SUM((td.gross_line_amount - td.pos_discount_amount)),2) AS total_amount
            FROM transaction_detail
              as td
              INNER JOIN customer_basket_scv_2017 AS cbs  ON
                td.av_transaction_id = cbs.transaction_id
            WHERE
              td.transaction_date >='{}'
            AND  
                td.transaction_date < '{}'
              AND cbs.customer_identifier IS NOT NULL
              AND customer_identifier IN {} 
              AND ( td.sku_id is NOT NULL OR td.line_object_description IN ('OMS Sales Tax',
'S6 Special Order T2 Sales Tax',
'S6 T2 Sales Tax',
'-Order T2 Sales Tax',
'OMS T2 Sales Tax',
'S6 Sales Tax',
'-Order Sales Tax',
'S6 Special Order Sales Tax'))
            GROUP BY cbs.customer_identifier
          ) a
      ) b
    ORDER BY RFM DESC, AmountPerWeek DESC, total_amount DESC
""".format(dt_start, dt_end,str(CUST_LIST)))
    

    
    start_time = time.time()

    trFrame = pd.read_sql_query(query, conn)

    conn.close()

    end_time = time.time()

    run_time = (end_time-start_time)/60

    print(run_time)
    return(trFrame)

In [118]:
CUST_LIST = ('9898420604', '161087221')

temp_DF = SQL_Query('2014/07/01','2017/07/26', 'Dhananjay.Kumar','password1!', CUST_LIST)

0.4189467827479045


In [126]:
(temp_DF.LastOrderDate -temp_DF.FirstOrderDate )/7

0   158 days 03:25:42.857142
1    11 days 13:42:51.428571
dtype: timedelta64[ns]

In [120]:
temp_DF.dtypes

customer_id               object
RFM                      float64
recency                    int64
frequency                  int64
T                          int64
order_count                int64
total_amount             float64
FirstOrderDate    datetime64[ns]
LastOrderDate     datetime64[ns]
dtype: object

# Declare our functions

In [29]:
def SQL_Query(start_date, end_date, UID, password, CUST_LIST ):

    
#Format our date inputs  
    i = str(start_date)
    try:
        dt_start = datetime.datetime.strptime(i, '%Y/%m/%d')
    except ValueError:
        print("Incorrect start format")

    i = str(end_date)    
    try:
        dt_end = datetime.datetime.strptime(i, '%Y/%m/%d')
    except ValueError:
        print("Incorrect start format")

#Formatting the dates for SQL 
    dt_start = str(dt_start) + str('.000')
    dt_end = str(dt_end) + str('.000')
    
#Establish our connection to the server 
    conn_string = 'DRIVER=/usr/local/lib/libtdsodbc.so;SERVER=wnj-datasizesql;PORT=1433;DATABASE=Data Analytics;UID={};PWD={}'.format(UID,password)
    conn = pyodbc.connect(conn_string)
    
#Run our sql query

    query = ("""SELECT  Customer_Profile.customerno, c.customer_id,
    c.RFM, c.recency, c.frequency,c.order_count, c.T,
    c.AmountPerWeek, c.total_amount,
    c.FirstOrderDate, c.LastOrderDate
    FROM
  (SELECT b.customer_id,
    round(b.recency + b.frequency/10 + b.AmountPerWeek/100, 2) AS RFM,
    b.recency, b.frequency,b.order_count, b.T,
    b.AmountPerWeek, b.total_amount,
    b.FirstOrderDate, b.LastOrderDate
    FROM
      (

        SELECT
          a.customer_id,
          a.recency,
          a.frequency,
          a.order_count,
          a.T,
          round(a.total_amount / a.frequency, 2) AS AmountPerWeek,
          a.total_amount,
          a.FirstOrderDate,
          a.LastOrderDate
        FROM
          (
            SELECT
              customer_id,
              count(DISTINCT cast(DATEPART(WK, transaction_date) AS VARCHAR) + cast(year(transaction_date) AS VARCHAR))
                AS frequency,
              datediff(WEEK, MIN(transaction_date), MAX(
                  transaction_date)) AS recency,
              datediff(WEEK, MIN(transaction_date),
                       '{}')    AS T,
              count(distinct(transaction_id)) as order_count,
              MIN(
                  transaction_date)  AS FirstOrderDate,
              MAX(
                  transaction_date)  AS LastOrderDate,
              round(sum(tender_amount_total),
                    2)               AS total_amount
            FROM transaction_header
            WHERE
              transaction_date >= '{}'
            AND  
              transaction_date < '{}' 
              AND customer_id IS NOT NULL
              AND customer_id IN {} 
            GROUP BY customer_id
          ) a
      ) b
    ) c
    inner join Customer_Profile ON c.customer_id = cast(Customer_Profile.customer_id AS int)
ORDER BY RFM DESC, AmountPerWeek DESC, total_amount DESC

    """.format(dt_end, dt_start, dt_end,str(CUST_LIST) ))

    start_time = time.time()

    trFrame = pd.read_sql_query(query, conn)

    conn.close()

    end_time = time.time()

    run_time = (end_time-start_time)/60

    print(run_time)
    return(trFrame)

In [11]:
def scoring_BG(customer_df, parameter_dictionary, weeks):

    start  = time.time()



    a = parameter_dictionary['a']
    alpha = parameter_dictionary['alpha']
    b= parameter_dictionary['b']
    r = parameter_dictionary['r']
    print(a, alpha, b, r)


    customer_df['period'] = str(i)+'_'+str(j)
    customer_df['BG_cond_purch_12'] = customer_df.apply(lambda q: clv_scoring_functions.BDF_conditional_expected_number_of_purchases_up_to_time(weeks,a,alpha, b, r, q['frequency'], q['recency'], q['T']), axis=1)
    customer_df['BG_prob_alive'] = customer_df.apply(lambda q: clv_scoring_functions.BDF_conditional_probability_alive(a,alpha, b, r, q['frequency'], q['recency'], q['T']), axis=1)
    customer_df['BG_prob_alive'] = customer_df.BG_prob_alive.astype(float)
    customer_df['BG_pred_spend'] = customer_df.BG_cond_purch_12*customer_df.AmountPerWeek




    end = time.time() 

    print("It took {} minutes to score {} customers with the following parameters \n {}".format(((end-start)/60),customer_df.shape[0], parameter_dictionary ))
    return(customer_df)      

In [12]:
#create a dataframe with our accuracy scores for each class of customers and one for the total 

def accuracy_score(comparison_frame):
    """
    Parameters 
    -----------
    comparison_frame - dataframe 
        A data frame with the predicted bins and the actual bins
    
    Returns
    --------
    data frame 
        A data frame with the accuracy score for each class along with population size and overall population 
        accuracy score. 
    """
        
    temp_df = pd.DataFrame()
    i=0
    for group in comparison_frame.custora_bins.unique():
        i+=1
        comparison_frame_sub = comparison_frame[comparison_frame.custora_bins==group]
        temp_df.at[i,'class']=group
        temp_df.at[i,'size']=comparison_frame_sub.shape[0]

        temp_df.at[i,'acc_score']= sklearn.metrics.accuracy_score(comparison_frame_sub.group, comparison_frame_sub.custora_bins)
        temp_df.at[i,'f1_score']= sklearn.metrics.f1_score(comparison_frame_sub.group, comparison_frame_sub.custora_bins, average='weighted')



    temp_df.at[i+1,'class']='All'
    temp_df.at[i+1,'size']=comparison_frame.shape[0]

    temp_df.at[i+1,'acc_score']= sklearn.metrics.accuracy_score(comparison_frame.group, comparison_frame.custora_bins)

    del comparison_frame_sub

    return(temp_df)

In [13]:
def custora_group_scoring(VS_df, VS_score_col):
    """
    Parameters 
    -----------
    VS_df - dataframe 
        the data frame with our scored customers (should have customer number
        or customer id - whatever we are going to use to merge this with the 
        custora results )
    VS_score_col - 'string'
        the column which contains our $ predictions for customers, include 
        quotation marks around the string
        
    
    Returns
    --------
    data frame 
        A data frame with a percentile rank, numerical rank and bins for 
        each customer (the bins are determined by custora's definition of 
        different customer classes, see their website for any changes)
    
    
    """
    
    VS_df['pred_spend_rank'] = VS_df[VS_score_col].rank()
    VS_df['pred_spend_perc'] = VS_df[VS_score_col].rank(pct=True)


    #these bins correspond to custoras categories. The dictionary corresponds to the ranges in the bins (in order)
    bins = np.array([0.0, 0.50, .75, .90, .95,.99,1.0001])
    bins_dict = {1:'low', 2:'medium', 3:'bronze', 4:'silver', 5:'gold', 6: 'platinum'}

    #we bin the percentile ranking using the bins articulated and then create a column that translates those 
    #bins to the custora group name 
    VS_df['pred_spend_bins'] = np.digitize(VS_df.pred_spend_perc, bins, right=False)
    VS_df['custora_bins'] = VS_df['pred_spend_bins'].replace(bins_dict)
    
    #print a sanity check to make sure the bins are correctly structured
    print("Please check to make sure that these bins make sense()\n",format(VS_df.custora_bins.value_counts()/VS_df.shape[0]))
    
    del VS_df['pred_spend_rank']
    del VS_df['pred_spend_perc']
    del VS_df['pred_spend_bins']
    
    return(VS_df)

    

In [14]:
global model_results_list
    
model_results_list = list()
 

def minimizationfunc(params):
    
    
       
    params = params
    
    parameters = {'a':params[0], "alpha" : params[1], 'b' :params[2], 'r':params[3]}

    
    
    test_df_scored = scoring_BG(sampled_cumulative_DF, parameters, 104)

    
    test_df_scored['pred_spend_bins'] = np.digitize(test_df_scored.BG_pred_spend, bin_separators, right=False)

    test_df_scored['custora_bins'] = test_df_scored['pred_spend_bins'].replace(bins_dict)

    
    #compare the bins for the customers on the columns 
    scored = (1 - sklearn.metrics.accuracy_score(test_df_scored.group, test_df_scored.custora_bins))
    print("This is your (1- accuracy score) {}".format(scored))
    scored = float(scored)
    
    model_results_list.append([parameters, scored])

    return(scored)
    
    


In [15]:

#create a list of pairwise dates to pass onto a SQL query that requires a specific format - ['2016/11/01', '2017/11/01']
#
def query_date(start_date, end_date, interval_months):



    date_list = []

    start_date = datetime.datetime.strptime(start_date, '%m/%d/%Y')
    end_date = datetime.datetime.strptime(end_date, '%m/%d/%Y')


    start_date = datetime.date(start_date.year, start_date.month, start_date.day)
    end_Date = datetime.date(end_date.year, end_date.month, end_date.day)

    current = start_date
    lag_month = start_date
    while lag_month < end_Date:
        current = lag_month
        current += relativedelta(months=interval_months)
        date_list.append([(lag_month).strftime('%Y/%m/%d'), (current).strftime('%Y/%m/%d')])
        lag_month += relativedelta(months=1)


    return(date_list)

# Bring In Our Data

## Bring in VS Transaction date summarized output -recency, frequency, age and total monetary value 

In [6]:
date_list= [['2018/01/01', '2018/04/01']]

In [7]:
date_list

[['2018/01/01', '2018/04/01']]

In [41]:
temp_DF

In [10]:
#create a dataframe with all our sql queries appended together with their data

#date_list = query_date('11/1/2017', '2/1/2019', 1 )

cumulative_DF = pd.DataFrame()

for i,j in date_list:
    temp_DF = SQL_Query(i,j, 'Dhananjay.Kumar','password1!')
    cumulative_DF = cumulative_DF.append(temp_DF)
    del temp_DF

    
#Print information that validates our query - find the first and last transaction for the new dataframe 

print("we have data from {} to {}.".format(cumulative_DF.FirstOrderDate.min(), cumulative_DF.LastOrderDate.max()))

38.80082781712214
we have data from 2016-04-01 00:00:00 to 2018-03-31 00:00:00.


In [16]:
#Optionally, save the dataframe so that we can double check the results later 

cumulative_DF.to_csv(r'/Users/sanjaygopinath/Documents/Data/cust_scored_2016_April_2018_April.csv', index=None)

In [17]:
cumulative_DF.head()

,customerno,customer_id,RFM,recency,frequency,order_count,T,AmountPerWeek,total_amount,FirstOrderDate,LastOrderDate,period,BG_cond_purch_12,BG_prob_alive,BG_pred_spend
0,714224203,9034412,122.05,104,13,18,105,1704.63,22160.16,2016-04-01,2018-03-27,2016/04/01_2018/04/01,12.400994,0.969637,21139.106873
1,9898909543,8197764,118.96,104,105,245,105,496.26,52107.51,2016-04-02,2018-03-29,2016/04/01_2018/04/01,88.641276,0.991413,43989.119748
2,501491851,18629588,118.77,101,24,126,102,1577.13,37851.11,2016-04-18,2018-03-28,2016/04/01_2018/04/01,22.095937,0.981789,34848.164632
3,9898528712,4445191,118.44,104,90,222,105,543.97,48957.53,2016-04-02,2018-03-30,2016/04/01_2018/04/01,76.246636,0.991255,41475.882776
4,9905214575,9609834,118.41,104,106,653,105,440.89,46734.57,2016-04-02,2018-03-30,2016/04/01_2018/04/01,89.467094,0.991417,39445.147114


In [13]:
#Let's remove anyone who has negative or zero value for the amount they spent 

cumulative_DF = cumulative_DF[cumulative_DF.AmountPerWeek > 0]


## Bring in Custora data

In [13]:
#import all our custora files from a specific file and then append the file name

#this is for spending predictions


path = r'/Users/sanjaygopinath/Documents/Data/CLV/Custora_Data/Machine_Learning/Spend/Jan_18' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))  

data = []

for csv in all_files:
    frame = pd.read_csv(csv)
    frame['filename'] = os.path.basename(csv)
    data.append(frame)

bigframe = pd.concat(data, ignore_index=True)

if 'Treatment Name' in bigframe.columns:
    del bigframe['Treatment Name']
    
#this creates a new column with the custora group portion of the file name. It has to be of the following format to 
# work (this is the format which is automatically downloaded from the custora app/website) - SG_Bronze_users_2018_01_09.csv
bigframe['group'] = bigframe['filename'].apply(lambda x: x.partition('_')[-1].partition('_')[0].lower())

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
bigframe.group.value_counts()

low         6654840
medium      3327420
bronze      1996452
silver       665484
gold         531175
platinum     132794
Name: group, dtype: int64

# Score our population

In [14]:
#manually input our parameters

                
#It took 3.3765572985013326 minutes to score 888193 customers with the following parameters 
#parameters = {'a': 0.06075756, 'alpha': 6.2382612304687521, 'b': 0.70402410888671718, 'r': 2.8112931823730491}
#This is your (1- accuracy score) 0.6128161334304594

#fourth iteration with 296064 customers
#parameters = {'a': 0.025867786259379202, 'alpha': 9.8203401747296937, 'b': 1.5017444877131725, 'r': 1.293332834267968}
#This is your (1- accuracy score) 0.46638902399481197

#third iteration with 100k sample size 
#parameters = {'a': 0.047582270378977554, 'alpha': 9.7527545777315225, 'b': 1.2970538830503067, 'r': 0.97965271560800504}
#This is your (1- accuracy score) 0.47375

#second iteration with 100k sample size 
#parameters = {'a': 0.060757568359,  'alpha': 6.2382612304687521, 'b': 0.70402410888671718, 'r': 2.8112931823730491}             

#first iteration with 10k sample size
#parameters = {'a':.060965087890624825, "alpha" : 7.2610668945312398, 'b' :0.75478155517578061, 'r':2.7994494934082095}

#iteration with sample size of 5507770
#parameters = {'a': 0.031435172173152087,'alpha': 6.0207392676930009,'b': 3.560652837671328,'r': 0.76827969369016613}
#This is your (1- accuracy score) 0.473750.3624563117196252
#parameters = {'a': 0.0314,'alpha': 6.021,'b': 3.56,'r': 0.768}

#old
parameters = {'a':.34, "alpha" : 5.44, 'b' :.52, 'r':2.66}

#create a list of our parameters so they can be referenced in our optimization function (it needs to be an array
# not a dictionary)

params = [(v) for k,v in parameters.items()]


In [15]:
#scoring the whole population 

cumulative_DF = scoring_BG(cumulative_DF, parameters, 104) 

0.34 5.44 0.52 2.66
It took 33.18724016745885 minutes to score 8885934 customers with the following parameters 
 {'a': 0.34, 'alpha': 5.44, 'b': 0.52, 'r': 2.66}


In [73]:
# Bin our data based on Custora's groupings

cumulative_DF = custora_group_scoring(cumulative_DF, 'BG_pred_spend')

Please check to make sure that these bins make sense()
 low         0.50
medium      0.25
bronze      0.15
silver      0.05
gold        0.04
platinum    0.01
Name: custora_bins, dtype: float64


In [74]:
#set the indices as your common shared customer id ('User ID'/'customerno'/'customer_ID')

#cumulative_DF['customerno'] = cumulative_DF['customerno'].astype(str)
#bigframe['User ID'] = bigframe['User ID'].astype(str)


#cumulative_DF.set_index(cumulative_DF['customerno'], inplace=True)

#bigframe.set_index(bigframe['User ID'], inplace=True)


#join the custora dataframe and our scored df  
comparison_frame = cumulative_DF.join(bigframe, how='inner')

#compare the bins for the customers on the bins 
sklearn.metrics.accuracy_score(comparison_frame.custora_bins, comparison_frame.group)

0.72682337860876545

In [76]:
sklearn.metrics.accuracy_score(comparison_frame.custora_bins, comparison_frame.group)

0.72682337860876545

In [81]:
bigframe.group.unique()

array(['bronze', 'low', 'platinum', 'medium', 'silver', 'gold'], dtype=object)

In [86]:
top_10_custora = bigframe[bigframe['group'].isin(['platinum', 'silver', 'gold'])]

In [87]:
top_10_custora.shape[0]/bigframe.shape[0]

0.09989754410168494

In [123]:
bigframe.shape[0]*.15

1996224.75

In [113]:
1329453-1330816.5

-1363.5

In [94]:
top_10_custora_list = list(top_10_custora.index)

In [110]:
top_10_custora.shape

(1329453, 5)

In [100]:
len(top_10_custora_list)

1329453

In [124]:
temp_DF = cumulative_DF.head(1996224)
#VS_top_10_list = list(temp_DF.index)
sum(temp_DF.index.isin(top_10_custora_list))/len(top_10_custora_list)

0.96090647807782603

In [106]:
sum(temp_DF.index.isin(top_10_custora_list))

1130903

In [107]:
1130903/1329453

0.8506528624930705

In [89]:
cumulative_DF.columns

Index(['customerno', 'customer_id', 'RFM', 'recency', 'frequency',
       'order_count', 'T', 'AmountPerWeek', 'total_amount', 'FirstOrderDate',
       'LastOrderDate', 'period', 'BG_cond_purch_12', 'BG_prob_alive',
       'BG_pred_spend', 'custora_bins'],
      dtype='object')

In [90]:
cumulative_DF.sort_values('BG_pred_spend', ascending=False, inplace=True)

In [37]:
comparison_frame.head()

,customerno,customer_id,RFM,recency,frequency,order_count,T,AmountPerWeek,total_amount,FirstOrderDate,...,period,BG_cond_purch_12,BG_prob_alive,BG_pred_spend,custora_bins,User ID,Email,Channels,filename,group
1000006,1000006,1898985,0.42,0,1,1,129,41.56,41.56,2015-07-31,...,2015/01/01_2018/01/17,0.200469,0.117842,8.331511,low,1000006,NaN,NaN,SG_Low_users_2018_01_09.csv,low
1000009,1000009,1921283,130.53,129,13,16,158,53.04,689.55,2015-01-05,...,2015/01/01_2018/01/17,8.549080,0.971967,453.443224,bronze,1000009,NaN,NaN,SG_Silver_users_2018_01_09.csv,silver
1000023,1000023,1516319,0.14,0,1,1,64,14.49,14.49,2016-10-28,...,2015/01/01_2018/01/17,1.152520,0.362466,16.700018,low,1000023,NaN,NaN,SG_Low_users_2018_01_09.csv,low
10000350601,10000350601,25651748,104.25,104,3,3,125,25.07,75.22,2015-08-24,...,2015/01/01_2018/01/17,3.232057,0.984948,81.027660,low,10000350601,marycoco777@gmail.com,NaN,SG_Medium_users_2018_01_09.csv,medium
1000066,1000066,2102293,0.27,0,1,1,24,26.98,26.98,2017-07-31,...,2015/01/01_2018/01/17,5.308388,0.773018,143.220320,medium,1000066,hornbird@optonline.net,NaN,SG_Medium_users_2018_01_09.csv,medium


In [48]:
comparison_frame = comparison_frame[comparison_frame.frequency >1]

In [78]:
comparison_frame.head()

,customerno,customer_id,RFM,recency,frequency,order_count,T,AmountPerWeek,total_amount,FirstOrderDate,...,period,BG_cond_purch_12,BG_prob_alive,BG_pred_spend,custora_bins,User ID,Email,Channels,filename,group
1000006,1000006,1898985,0.42,0,1,1,129,41.56,41.56,2015-07-31,...,2015/01/01_2018/01/17,0.000028,0.000012,0.001146,low,1000006,NaN,NaN,SG_Low_users_2018_01_09.csv,low
1000009,1000009,1921283,130.53,129,13,16,158,53.04,689.55,2015-01-05,...,2015/01/01_2018/01/17,5.687281,0.633509,301.653389,bronze,1000009,NaN,NaN,SG_Silver_users_2018_01_09.csv,silver
1000023,1000023,1516319,0.14,0,1,1,64,14.49,14.49,2016-10-28,...,2015/01/01_2018/01/17,0.000530,0.000137,0.007684,low,1000023,NaN,NaN,SG_Low_users_2018_01_09.csv,low
10000350601,10000350601,25651748,104.25,104,3,3,125,25.07,75.22,2015-08-24,...,2015/01/01_2018/01/17,2.795534,0.732928,70.084030,medium,10000350601,marycoco777@gmail.com,NaN,SG_Medium_users_2018_01_09.csv,medium
1000066,1000066,2102293,0.27,0,1,1,24,26.98,26.98,2017-07-31,...,2015/01/01_2018/01/17,0.023556,0.003156,0.635554,low,1000066,hornbird@optonline.net,NaN,SG_Medium_users_2018_01_09.csv,medium


In [80]:
bigframe.columns

Index(['User ID', 'Email', 'Channels', 'filename', 'group'], dtype='object')

In [ ]:
custora_top_10 = bigframe['customera']

In [26]:
comparison_frame.custora_bins.value_counts()

low         5547002
medium      2760398
bronze      1638384
silver       537528
gold         425103
platinum     107125
Name: custora_bins, dtype: int64

In [27]:
comparison_frame.group.value_counts()

low         4380954
medium      3312101
bronze      1993564
silver       665099
gold         531037
platinum     132785
Name: group, dtype: int64

In [65]:
score_frame4 = accuracy_score(comparison_frame)

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [70]:
score_frame4

,class,size,acc_score,f1_score
1,low,5547237.0,0.732257,0.619078
2,bronze,1642255.0,0.521144,0.357088
3,medium,2763574.0,0.549484,0.389720
4,gold,420794.0,0.673455,0.542043
5,silver,536813.0,0.341113,0.173525
6,platinum,104867.0,0.847102,0.776981
7,All,11015540.0,0.634715,NaN


In [53]:
score_frame2

,class,size,acc_score,f1_score
1,bronze,1590995.0,0.733862,0.621219
2,medium,2394528.0,0.679025,0.549218
3,low,1633467.0,0.755637,0.650462
4,silver,538358.0,0.596196,0.445371
5,platinum,109739.0,0.915718,0.875431
6,gold,429322.0,0.825495,0.746583
7,All,6696409.0,0.717352,NaN


In [67]:
score_frame

,class,size,acc_score,f1_score
1,low,5547486.0,0.754505,0.648933
2,bronze,1630230.0,0.730832,0.617178
3,medium,2754962.0,0.672330,0.540596
4,silver,542537.0,0.593812,0.442477
5,platinum,109809.0,0.915144,0.874595
6,gold,430516.0,0.823242,0.743432
7,All,11015540.0,0.726823,NaN


In [32]:
comparison_frame['custora_bins'].groupby(comparison_frame['group']).value_counts().to_frame().to_csv('bins.csv')

In [ ]:
sampled_cumulative_DF

In [ ]:
#this establishes the size of our sample data frame which we use for our optimization function 
#Check the minimization function, it references our sampled_cumulative_DF

sampled_cumulative_DF = comparison_frame.sample(frac=.5)
sampled_cumulative_DF.shape

In [ ]:
bins = np.array([0.0, 0.50, .75, .90, .95,.99,1.0001])
bins_dict = {1:'low', 2:'medium', 3:'bronze', 4:'silver', 5:'gold', 6: 'platinum'}


#use this on the fully scored data set and then use it for bins on the samples 
bin_separators = [(np.percentile(cumulative_DF.BG_pred_spend, int(x*100))) for x in bins]

#we reset the top and bottom of the bins so that they encompass all values (think about the percentiles, it is 
# easier to do this than to hope that there aren't any rounding issues)
bin_separators[0]= 0
bin_separators[6] = cumulative_DF.BG_pred_spend.max()+1

#use this if you are using a sample because otherwise the demarcation of bins, in absolute $ value, won't make sense 
#VS_df['pred_spend_bins'] = np.digitize(VS_df.pred_spend_perc, bins, right=False)

#the above changes to this - 
#VS_df['pred_spend_bins'] = np.digitize(VS_df.BG_cond_purch_12, bins, right=False)



In [ ]:
#run our minimization function

res = minimize(minimizationfunc, params, method='Nelder-Mead')

In [ ]:

#Sort our model results and then take the first one. Since we are minimzing and we are sorting in ascending order
# we will have the smallest (most accurate) score first and we want to take the parameters from that)
model_results_list = sorted(model_results_list, key=operator.itemgetter(1))   # Ascending order
parameters = model_results_list[0][0]
params = [(v) for k,v in parameters.items()]

In [ ]:
params

In [ ]:
model_results_list

In [ ]:
sampled_cumulative_DF

In [ ]:
params